### LangChain's `PyPDFLoader`, `PyMuPDFLoader`

In [78]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader # 솓도 최적화가 되어 있는 로더 
from langchain_ollama import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(
    base_url = "http://localhost:11434",
    model = "gemma3:latest",
    temperature=0.1
)

llm.model

FILE_PATH = "./data/Context-Aware Named Entity Recognition for Neologism Detection- A Deep Learning Approach to Bridging Generational Language Gaps.pdf"


In [79]:
%%time

loader = PyPDFLoader(FILE_PATH)
docs = loader.load()

CPU times: total: 438 ms
Wall time: 455 ms


In [80]:
for doc in docs[:10]:
    print(doc.page_content)

Context-Aware Named Entity Recognition 
for Neologism Detection: A Deep Learning 
Approach to Bridging Generational 
Language Gaps  
Junhaeng Lee,  HyunYoung Oh , SangWoo Park, and Young -myoung Kang*  
Abstract—The rapid proliferation of neologisms in online and social communication has intensified linguistic 
disparities across generations, posing challenges for timely and accurate neologism detection. Conventional 
dictionary- and rule-based methods struggle to ca pture the dynamic, context -dependent nature of evolving 
expressions, limiting their practical applicability. This study proposes a deep learning -based, context-aware 
Named Entity Recognition (NER) model designed to effectively identify neologisms and mitigate generational 
language gaps. The model leverages KoELECTRA, a pre -trained Korean language model optimized for 
contextual understanding,  and is evaluated on large -scale datasets with varying entity distributions. 
Experimental results demonstrate that the propo

In [81]:
%%time
loader2 = PyMuPDFLoader(FILE_PATH)

docs2 = loader2.load()

CPU times: total: 31.2 ms
Wall time: 32.5 ms


In [82]:
for doc in docs2:
    print(doc.page_content)

Context-Aware Named Entity Recognition 
for Neologism Detection: A Deep Learning 
Approach to Bridging Generational 
Language Gaps 
Junhaeng Lee, HyunYoung Oh, SangWoo Park, and Young-myoung Kang* 
Abstract—The rapid proliferation of neologisms in online and social communication has intensified linguistic 
disparities across generations, posing challenges for timely and accurate neologism detection. Conventional 
dictionary- and rule-based methods struggle to capture the dynamic, context-dependent nature of evolving 
expressions, limiting their practical applicability. This study proposes a deep learning-based, context-aware 
Named Entity Recognition (NER) model designed to effectively identify neologisms and mitigate generational 
language gaps. The model leverages KoELECTRA, a pre-trained Korean language model optimized for 
contextual understanding, and is evaluated on large-scale datasets with varying entity distributions. 
Experimental results demonstrate that the proposed model a

In [83]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len, # 텍스트 길이 계산 함수 지정 
    is_separator_regex=False 
)

texts = text_splitter.split_documents(docs)

In [98]:
for text in texts[:10]:
    print(text.page_content[:300])

Context-Aware Named Entity Recognition 
for Neologism Detection: A Deep Learning 
Approach to Bridging Generational 
Language Gaps  
Junhaeng Lee,  HyunYoung Oh , SangWoo Park, and Young -myoung Kang*  
Abstract—The rapid proliferation of neologisms in online and social communication has intensified
contextual understanding,  and is evaluated on large -scale datasets with varying entity distributions. 
Experimental results demonstrate that the proposed model achieves robust neologism detection 
performance, with an F1 -score of 0.89 under real -world imbalanced conditions and up to 0.92 in cont
Recognition, Natural Language Processing. 
――――――――――――――――――――――――――――――――――――――――――――――――――――――――― 
1 INTRODUCTION
In contemporary society, conflicts driven by na-
tional, racial, religious, and generational divides are 
escalating globally. Among these, generational con-
flict has become a critic
The proliferation of internet -based platforms 
and social media has significantly accelerated th

In [85]:
len(texts)

36

In [86]:
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    '''단계적으로 생각하세요. 당신은 NLP 전문가입니다. {docs}를 참고해서 전체적으로 어떤 내용인지 요약해주면 됩니다.
    또한 핵심 기술을 설명해주면 됩니다.'''
])

chain = prompt | llm | StrOutputParser()

In [87]:
print(chain.get_graph().draw_ascii())

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOllama |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


In [88]:
print(chain.invoke(texts))

## 전체 요약

이 문서는 **네오로그리즘(Neologism)을 탐지하기 위한 문맥 인지된 Named Entity Recognition (NER) 모델**에 대한 연구 논문입니다. 특히, 네오로그리즘을 정확하게 식별하는 데 중점을 두고 있으며, 이를 위해 딥러닝 기반의 모델을 활용합니다. 연구팀은 다양한 딥러닝 모델(LSTM-CRF, BERT 등)을 비교 분석하고, 문맥 정보를 효과적으로 활용하여 NER 성능을 향상시키는 방법을 제시합니다. 또한, 연구 결과는 네오로그리즘 탐지 분야에 기여할 수 있는 잠재력을 보여줍니다.

## 핵심 기술

*   **Named Entity Recognition (NER):**  텍스트에서 특정 유형의 개체(예: 사람, 장소, 조직 등)를 식별하는 기술입니다. 이 논문에서는 네오로그리즘을 포함한 개체를 식별하는 데 사용됩니다.
*   **딥러닝 (Deep Learning):**  복잡한 패턴을 학습하기 위해 인공 신경망을 깊게 쌓아 올린 기술입니다. 이 논문에서는 LSTM-CRF, BERT와 같은 딥러닝 모델을 활용하여 NER 성능을 향상시킵니다.
    *   **LSTM (Long Short-Term Memory):**  순환 신경망(RNN)의 한 종류로, 장기 의존성을 학습하는 데 효과적입니다.
    *   **CRF (Conditional Random Field):**  시퀀스 레이블링 문제 해결에 사용되는 모델로, 시퀀스 내의 레이블 간의 의존성을 고려합니다.
    *   **BERT (Bidirectional Encoder Representations from Transformers):**  Google에서 개발한 트랜스포머 기반의 사전 훈련된 언어 모델입니다. 문맥 정보를 효과적으로 활용하여 다양한 NLP 작업에서 뛰어난 성능을 보입니다.
*   **문맥 인지 (Contextual Awareness):**  단어 자체의 의미뿐만 아니라, 주변 문맥을 고려하여 개체를 식별하는 기술입니다. 이 논문에서는 BERT

### Pydantic으로 모델의 응답을 구조화

In [89]:
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import date
from langchain_core.output_parsers import StrOutputParser

class PaperSummary(BaseModel):
    title: str = Field(..., description="논문의 제목")
    authors: List[str] = Field(..., description="논문의 저자 목록")
    publication_date: Optional[str] = Field(None, description="논문이 출판된 날짜 또는 연도 (YYYY-MM-DD 또는 YYYY)")
    keywords: List[str] = Field(..., description="논문의 핵심 키워드 목록")
    abstract_summary: str = Field(..., description="논문 초록의 핵심 내용을 요약")
    main_contributions: List[str] = Field(..., description="논문이 기여한 주요 내용들을 목록으로 요약")
    methodology: str = Field(..., description="논문에서 사용된 주요 방법론을 간결하게 설명")
    results: str = Field(..., description="논문 실험의 주요 결과를 요약")

parser = StrOutputParser(pydantic_object=PaperSummary)

In [96]:
from langchain_core.prompts import PromptTemplate

# Pydantic 모델의 JSON 스키마를 가져옵니다.
json_schema = PaperSummary.model_json_schema()

# 프롬프트 템플릿 정의
prompt = PromptTemplate(
    template="""당신은 논문을 분석하고 요약하는 전문가입니다.
    사용자가 제공한 논문 내용을 분석하여, 아래에 제공된 JSON 스키마 형식에 맞춰서 응답을 생성해 주세요.

    <논문 내용>
    {paper_content}
    </논문 내용>

    <출력 JSON 스키마>
    {json_schema}
    </출력 JSON 스키마>

    주의사항:
    - 논문 내용에서 핵심 정보를 추출하여 JSON 객체를 완성하세요.
    - JSON 형식으로만 응답해야 하며, 다른 부가적인 설명은 절대 포함하지 마세요.
    - 모든 필드는 스키마에 정의된 데이터 타입과 일치해야 합니다.
    """,
    input_variables=["paper_content"],
    partial_variables={"json_schema": json_schema}
)

In [94]:
summary_chain = prompt | llm | parser 
print(summary_chain.get_graph().draw_ascii())

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOllama |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


In [97]:
## Trouble Shoothing이 발생하는데, 이는 LLM의 응답이 사전에 정의된 형식과 일치하지 않기 때문이다. 
print(summary_chain.invoke(texts))

```json
{
  "논문 제목": "Context-Aware Named Entity Recognition for Neologisms",
  "저자": [
    "Erin Espriu",
    "Junhaeng Lee",
    "HyunYoung Oh",
    "SangWoo Park",
    "Young-myoung Kang"
  ],
  "주요 내용": "본 논문은 새로운 단어(neologisms)를 인식하는 데 초점을 맞춘 Named Entity Recognition (NER) 기술을 제안합니다.  특히, 언어 사용자의 세대 간 차이를 고려하여 NER 모델을 개선하는 방법을 제시합니다.  논문은 다양한 NER 모델(LSTM-CRF, BERT 등)을 활용하고,  세대별 언어 사용 패턴을 학습하여 모델 성능을 향상시키는 방법을 설명합니다.  또한,  NER 모델의 성능을 평가하기 위한 실험 결과를 제시합니다.",
  "핵심 기술": [
    "Named Entity Recognition (NER)",
    "LSTM-CRF 모델",
    "BERT 모델",
    "세대별 언어 패턴 학습"
  ],
  "연구 배경": "새로운 단어(neologisms)는 언어 변화의 중요한 지표이며,  NER 모델이 이러한 변화를 정확하게 인식하는 것이 중요합니다.  특히,  세대 간 언어 사용 패턴의 차이를 고려하지 않으면 NER 모델의 성능이 저하될 수 있습니다."
}
```



### Json 데이터 생성 

In [67]:
prompt = """\
Can you create 100 city-based store locations for a fictitious company. The cities should be the kind of location where \
you might find a big box store like Target or Walmart. They should be located in the USA, or Europe, or Asia.

Provide your response in a list. Only return the list. Do not make any additional comments.\
"""

In [68]:
locations = llm.invoke(prompt).content

In [69]:
locations

'1. Atlanta, GA\n2. Baltimore, MD\n3. Boston, MA\n4. Buffalo, NY\n5. Charlotte, NC\n6. Chicago, IL\n7. Cincinnati, OH\n8. Cleveland, OH\n9. Columbus, OH\n10. Colorado Springs, CO\n11. Corpus Christi, TX\n12. Dallas, TX\n13. Denver, CO\n14. Des Moines, IA\n15. Detroit, MI\n16. El Paso, TX\n17. Fort Wayne, IN\n18. Fresno, CA\n19. Greensboro, NC\n20. Houston, TX\n21. Jacksonville, FL\n22. Kansas City, MO\n23. Las Vegas, NV\n24. Louisville, KY\n25. Memphis, TN\n26. Milwaukee, WI\n27. Minneapolis, MN\n28. Nashville, TN\n29. New Orleans, LA\n30. Newark, NJ\n31. Orlando, FL\n32. Oklahoma City, OK\n33. Oakland, CA\n34. Omaha, NE\n35. Orlando, FL\n36. Philadelphia, PA\n37. Phoenix, AZ\n38. Pittsburgh, PA\n39. Portland, OR\n40. Raleigh, NC\n41. Richmond, VA\n42. Riverside, CA\n43. Sacramento, CA\n44. Saint Louis, MO\n45. San Antonio, TX\n46. San Diego, CA\n47. San Francisco, CA\n48. San Jose, CA\n49. Santa Ana, CA\n50. Seattle, WA\n51. Shreveport, LA\n52. Sioux Falls, SD\n53. Stockton, CA\n54. S

In [70]:
raw_locations = locations.strip().split("\n")
raw_locations

['1. Atlanta, GA',
 '2. Baltimore, MD',
 '3. Boston, MA',
 '4. Buffalo, NY',
 '5. Charlotte, NC',
 '6. Chicago, IL',
 '7. Cincinnati, OH',
 '8. Cleveland, OH',
 '9. Columbus, OH',
 '10. Colorado Springs, CO',
 '11. Corpus Christi, TX',
 '12. Dallas, TX',
 '13. Denver, CO',
 '14. Des Moines, IA',
 '15. Detroit, MI',
 '16. El Paso, TX',
 '17. Fort Wayne, IN',
 '18. Fresno, CA',
 '19. Greensboro, NC',
 '20. Houston, TX',
 '21. Jacksonville, FL',
 '22. Kansas City, MO',
 '23. Las Vegas, NV',
 '24. Louisville, KY',
 '25. Memphis, TN',
 '26. Milwaukee, WI',
 '27. Minneapolis, MN',
 '28. Nashville, TN',
 '29. New Orleans, LA',
 '30. Newark, NJ',
 '31. Orlando, FL',
 '32. Oklahoma City, OK',
 '33. Oakland, CA',
 '34. Omaha, NE',
 '35. Orlando, FL',
 '36. Philadelphia, PA',
 '37. Phoenix, AZ',
 '38. Pittsburgh, PA',
 '39. Portland, OR',
 '40. Raleigh, NC',
 '41. Richmond, VA',
 '42. Riverside, CA',
 '43. Sacramento, CA',
 '44. Saint Louis, MO',
 '45. San Antonio, TX',
 '46. San Diego, CA',
 '47

In [73]:
locations_json = [{"location": location.split(". ")[1]} for location in raw_locations]
locations_json

[{'location': 'Atlanta, GA'},
 {'location': 'Baltimore, MD'},
 {'location': 'Boston, MA'},
 {'location': 'Buffalo, NY'},
 {'location': 'Charlotte, NC'},
 {'location': 'Chicago, IL'},
 {'location': 'Cincinnati, OH'},
 {'location': 'Cleveland, OH'},
 {'location': 'Columbus, OH'},
 {'location': 'Colorado Springs, CO'},
 {'location': 'Corpus Christi, TX'},
 {'location': 'Dallas, TX'},
 {'location': 'Denver, CO'},
 {'location': 'Des Moines, IA'},
 {'location': 'Detroit, MI'},
 {'location': 'El Paso, TX'},
 {'location': 'Fort Wayne, IN'},
 {'location': 'Fresno, CA'},
 {'location': 'Greensboro, NC'},
 {'location': 'Houston, TX'},
 {'location': 'Jacksonville, FL'},
 {'location': 'Kansas City, MO'},
 {'location': 'Las Vegas, NV'},
 {'location': 'Louisville, KY'},
 {'location': 'Memphis, TN'},
 {'location': 'Milwaukee, WI'},
 {'location': 'Minneapolis, MN'},
 {'location': 'Nashville, TN'},
 {'location': 'New Orleans, LA'},
 {'location': 'Newark, NJ'},
 {'location': 'Orlando, FL'},
 {'location': 

In [74]:
import json 
import os 

def write_jsonl(list_of_dict, file_name):
    with open(file_name, "w") as file:
        for dictionary in list_of_dict:
            file.write(json.dumps(dictionary) + "\n")
            
write_jsonl(locations_json, "data/store_locations.jsonl")